# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [120]:
# import libraries
import pandas
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import LinearSVC
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
import re


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/xweng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/xweng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/xweng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [96]:
# load data from database
engine = create_engine('sqlite:///pipeline.db')
df = pandas.read_sql_table(table_name='disaster',con=engine)
df.drop(['id'],inplace=True,axis=1)
X = df['message']
Y = df.loc[:, ~df.columns.isin(['message','original','genre'])] 

### 2. Write a tokenization function to process your text data

In [97]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    tokensWithoutStop = [w for w in tokens if w not in stopwords.words("english")]
    stemmedTokens = [PorterStemmer().stem(w) for w in tokensWithoutStop]
    lemmedTokens = [WordNetLemmatizer().lemmatize(w) for w in stemmedTokens]
    
    return lemmedTokens

In [98]:
# initialize tf-idf vectorizer object
#vectorizer = TfidfVectorizer()

In [99]:
# initialize count vectorizer object
#vect = CountVectorizer(tokenizer=tokenize)

In [100]:
# get counts of each token (word) in text data
#X_vec = vect.fit_transform(X.message)

In [101]:
# initialize tf-idf transformer object
#transformer = TfidfTransformer(smooth_idf=False)

In [102]:
# use counts from count vectorizer results to compute tf-idf values
#X_tfidf = transformer.fit_transform(X_vec)

In [103]:
# convert sparse matrix to numpy array to view
#X_tfidf.toarray()

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [104]:
#decision_tree_classifier = DecisionTreeClassifier()

In [105]:
#pipeline = MultiOutputClassifier(OneVsRestClassifier(LinearSVC()))

In [106]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC())))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [107]:
pipeline.get_params()


{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1e03761ae8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                                           class_weight=None,
                                                                           dual=True,
                                                                           fit_intercept=True,
                                  

In [108]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [110]:
pipeline.fit(X_train,y_train)

/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=OneVsRestClassifier(estimator=LinearSVC(C=1.0,
                                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [50]:
def evaluate_model(model, X_test, Y_test):
    '''
    Function to generate classification report on the model
    Input: Model, test set ie X_test & y_test
    Output: Prints the Classification report
    '''
    y_pred = model.predict(X_test)
    for i, col in enumerate(y_test):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [111]:
evaluate_model(pipeline,X_test,y_test)

related
              precision    recall  f1-score   support

           0       0.32      0.13      0.19      2046
           1       0.77      0.91      0.83      6545
           2       0.20      0.01      0.03        69

    accuracy                           0.72      8660
   macro avg       0.43      0.35      0.35      8660
weighted avg       0.65      0.72      0.67      8660

request
              precision    recall  f1-score   support

           0       0.85      0.95      0.89      7164
           1       0.41      0.19      0.26      1496

    accuracy                           0.81      8660
   macro avg       0.63      0.57      0.58      8660
weighted avg       0.77      0.81      0.78      8660

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8619
           1       0.00      0.00      0.00        41

    accuracy                           1.00      8660
   macro avg       0.50      0.50      0.50      866

/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      8599
           1       0.00      0.00      0.00        61

    accuracy                           0.99      8660
   macro avg       0.50      0.50      0.50      8660
weighted avg       0.99      0.99      0.99      8660

hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8565
           1       0.00      0.00      0.00        95

    accuracy                           0.99      8660
   macro avg       0.49      0.50      0.50      8660
weighted avg       0.98      0.99      0.98      8660

shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8618
           1       0.00      0.00      0.00        42

    accuracy                           1.00      8660
   macro avg       0.50      0.50      0.50      8660
weighted avg       0.99      1.00      0.99      8660

aid

### 6. Improve your model
Use grid search to find better parameters. 

In [116]:
parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__estimator__max_iter': [50, 100,500,1000], 
              'clf__estimator__estimator__dual': [True, False]} 

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [118]:
cv.fit(X_train, y_train)
evaluate_model(cv, X_test, y_test)

/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 i

/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0 is present in all training examples.
  str(classes[c]))
/Users/xweng/anaconda3/lib/python3.7/site-packag

/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of ite

related
              precision    recall  f1-score   support

           0       0.34      0.11      0.16      2046
           1       0.76      0.94      0.84      6545
           2       0.25      0.01      0.03        69

    accuracy                           0.73      8660
   macro avg       0.45      0.35      0.34      8660
weighted avg       0.66      0.73      0.67      8660

request
              precision    recall  f1-score   support

           0       0.84      0.96      0.90      7164
           1       0.40      0.15      0.21      1496

    accuracy                           0.82      8660
   macro avg       0.62      0.55      0.55      8660
weighted avg       0.77      0.82      0.78      8660

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8619
           1       0.00      0.00      0.00        41

    accuracy                           1.00      8660
   macro avg       0.50      0.50      0.50      866

/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8565
           1       0.00      0.00      0.00        95

    accuracy                           0.99      8660
   macro avg       0.49      0.50      0.50      8660
weighted avg       0.98      0.99      0.98      8660

shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8618
           1       0.00      0.00      0.00        42

    accuracy                           1.00      8660
   macro avg       0.50      0.50      0.50      8660
weighted avg       0.99      1.00      0.99      8660

aid_centers
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      8564
           1       0.00      0.00      0.00        96

    accuracy                           0.99      8660
   macro avg       0.49      0.50      0.50      8660
weighted avg       0.98      0.99      0.98      8660

o

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [121]:
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('best', TruncatedSVD()),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

In [122]:
pipeline2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                   dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                   lowercase=True, max_df=1.0, max_features=None, min_df=1,
                   ngram_range=(1, 1), preprocessor=None, stop_words=None,
                   strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                   tokenizer=<function tokenize at 0x1e03761ae8>, vocabulary=None)),
  ('best', TruncatedSVD(algorithm='randomized', n_components=2, n_iter=5,
                random_state=None, tol=0.0)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                          class_weight=None,
                                                          criterion='gini',
                                            

In [123]:
pipeline2.fit(X_train, y_train)

/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/xweng/anaconda3/lib/python3.7/site-pa

/Users/xweng/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [ ]:
evaluate_model(pipeline2, X_test, y_test)


### 9. Export your model as a pickle file

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv2, fpipeline2

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.